# TODO: incomplete, basically all copied code source at this point.
## Pubmed search on unrelated topic

Can build query at pubmed central.

http://www.ncbi.nlm.nih.gov/pmc/advanced

- 69992 pubmed ids
(((infectious disease) AND "free full text"[Filter]) AND ("2011/01/04"[Publication Date] : "2016/01/04"[Publication Date])) AND humans[MeSH Terms]

(((infectious disease) AND "free full text"[Filter]) AND ("2011/01/04"[PDat] : "2016/01/04"[PDat])) AND humans[MeSH Terms]

"Infectious disease"[All Fields] AND ("loattrfree full text"[sb] AND "2011/01/04"[PDat] : "2016/01/04"[PDat] AND "humans"[MeSH Terms])

## Create plain text markup of annotations.
1 - Use medic to generate / manage plain text abstracts.
2 - Use spacy to split the medic abstract sentences into 1 line per sentence.
3 -  Use shell script to ease process of manually applying tagging markup from extracted highlights to the plain text sentence split.
4 - Manually apply tagging markup to abstracts missed for some reason in 3

## save abstracts in medic

In [12]:
import io
import codecs
from spacy.en import English
nlp = English(parser=True) # so we can sentence parse

### Start postgres if necessary

In [14]:
# createdb medline
!pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start

server starting


In [3]:
wkdir = '/Users/ccarey/Documents/Projects/NAMI/rdoc'
%cd $wkdir

/Users/ccarey/Documents/Projects/NAMI/rdoc


## All our pubmed based annotation pdfs

In [16]:
# pubmed ids from a subset of file names
%cd ./pdfs/all_pdfs_annotated_pmid_names
pmids = !ls * | sed 's/[0_].*\([0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]\)_.*.pdf/\1/' | sed '/NA/d'
nas = !ls * | sed 's/[0_].*\([0-9][0-9][0-9][0-9][0-9][0-9][0-9][0-9]\)_.*.pdf/\1/' | sed '/NA/!d'
pmids = set(pmids)
print('------{} pubmed ids for medic ----:'.format(len(pmids)))
pmids = ' '.join(pmids)
cmd = 'medic update {}'.format(pmids)
print(pmids)

[Errno 2] No such file or directory: './pdfs/all_pdfs_annotated_pmid_names'
/Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/all_pdfs_annotated_pmid_names
------74 pubmed ids for medic ----:
23088207 25280468 23744445 25348131 23143607 24376698 24933724 25774613 24870123 21957257 24388670 25773639 21849230 24725811 25740534 25734385 25197810 25126038 25834059 20695690 23904684 23503620 25788679 23647728 23709163 24980898 23957953 25913552 25142564 24806675 25258728 24023823 20685988 22379245 23646134 24333377 24293773 24231418 24116095 21319926 24045586 24770625 21699821 24101292 24285346 24804717 25160677 24333745 24359877 24511281 23928891 23954763 22575329 25154749 25036160 25898427 22438994 22379238 20815182 23452958 21531705 24470693 23941878 23083918 23074247 21613467 24740391 25136085 25017671 25261920 23622762 23558179 22447249 25126029


In [18]:
print('------ medic command ------')
print(cmd)
print('------ medic warnings etc ------') # seems fine ignoring sqlalchemy warning.

#! $cmd

------ medic command ------
medic update 23088207 25280468 23744445 25348131 23143607 24376698 24933724 25774613 24870123 21957257 24388670 25773639 21849230 24725811 25740534 25734385 25197810 25126038 25834059 20695690 23904684 23503620 25788679 23647728 23709163 24980898 23957953 25913552 25142564 24806675 25258728 24023823 20685988 22379245 23646134 24333377 24293773 24231418 24116095 21319926 24045586 24770625 21699821 24101292 24285346 24804717 25160677 24333745 24359877 24511281 23928891 23954763 22575329 25154749 25036160 25898427 22438994 22379238 20815182 23452958 21531705 24470693 23941878 23083918 23074247 21613467 24740391 25136085 25017671 25261920 23622762 23558179 22447249 25126029
------ medic warnings etc ------


In [19]:
abstracts_m = !medic --error write --format tsv ALL 2> /dev/null | cut -f1,3

In [20]:
abstracts = []
type(abstracts)
with io.BytesIO(abstracts_m.n) as a:
    for line in a.readlines():
        pmid, abst = line.split('\t')
        abstracts.append({pmid: abst})

In [27]:
def spacy_split_sentences(text):
    sentences = []
    #doc = nlp(text.decode('utf8')) #"This is a sentence. Here's another...".decode('utf8'))
    doc = nlp(text) #"This is a sentence. Here's another...".decode('utf8'))
    for span in doc.sents:
        #sentences.append(u''.join(doc[i].string for i in range(span.start, span.end)).encode('utf-8').strip())
        sentences.append(''.join(doc[i].string for i in range(span.start, span.end)))#.strip())
    sentences = '\n'.join(sentences)
    sentences = sentences + '\n'
    return(sentences)

## save sentence split abstract for each file as .org file for easy editing in emacs.

- .txt files are files with annotations resulting from the pdf parsing of highlighted text
- .org files we create here.

For all abstracts:
- open a .org file of that abstract.
- open the annotations .txt file for that abstract.
- Edit the .org to apply tags indicating the annotations.
- Save the edited .org as a .txt file in an output directory.
- Remove the local .org file

Then, for each remaining .org file:
- cross-check any remaining .org files to see if there were .txt annotations that were missed.
  ( due to pattern matching errors, or maybe it was IRRELEVANT, MISCLASSIFIED etc. )
- if annotations were found, edit the corresponding .org and save .txt file to same output directory as above.

In [ ]:
%cd '/Users/ccarey/Documents/Projects/NAMI/rdoc/results'
%ls
# cleanup if we are testing
# ! rm *.org

In [28]:
for a in abstracts:
    fout = a.keys()[0] + '.org'
    with codecs.open(fout, 'w', 'utf-8') as f:
        body = a.values()[0].decode('utf-8')
        body_text = spacy_split_sentences(body)
        f.write(body_text)

## apply edits one pair of files at a time

BASH: Open the files in pairs for easy editing.

Note, not every [annotator].org file will have a matching .txt annotation.
+ 1) We are just working through files in the ./results/[annotator]/ folders
+ 2) Not every .org file had an equivalent annotation by the person.
+ 3) some files needed completely manual annotations. and were not in the ./results[annotator]/ folders


### edit abstracts per annotations
Create temporary files in current directory

for each file:
- edit abstracts in emacs
- copy results into subject/who/*.txt

Problem files:
Conclusions not included in abstract in 
  + 23744445

Different versions? (fully spelled out abbrevs from pubmed).
  + 24285346
  
A PMID Mislabeled as 23941878 corrected everywhere to 25258728

Then we have 25258728 twice, once for AR03 and once for AR04. It is unclear if it should have been in AR04, so delete those, keep only AR03_25258728.

In [29]:
! for f in *[0-9].org; do cp $f ${f/./_jl.}; done

JL: Check how many stranded .org we have

In [76]:
stranded = !ls *_jl.org | sed 's/.*\([0-9]\{8\}\).*/\1/'
print(len(stranded))
print(stranded)

1
['ls: *_jl.org: No such file or directory']


JL: check how many processed annotations we had

In [56]:
processed=!ls ./annotations_processed/*/jl/*.txt | sed 's/.*\([0-9]\{8\}\).*/\1/'
len(processed)

41

JL: Check how many total possible annotations we could have had.

In [57]:
possible=!ls ../pdfs/all_pdfs_annotated_pmid_names/*jl* | sed 's/.*\([0-9]\{8\}\).*/\1/'
len(possible)

50

Manually process the positives amongst the 8 that correspond to stranded .org.

41 + 8 = 49.

The remaining 1 to get to full 50 possible was not a pubmed document.

So we manually processed the straightforward annotations:

'_02_AP07_02_24804717_jl_ANNOTATED.pdf',  '_02_LS05_02_20685988_jl_ANNOTATED.pdf',
'_02_AR00_03_25788679_jl_ANNOTATED.pdf', '_03_AR03_24806675_jl_ANNOTATED.pdf'

and manually the irrelevant as we did for other normal irrelevants...

'_02_LS00_03_25913552_jl_IRRELEVANT.pdf',

## TODO: Still need to process the IRRELEVANT and MISCLASSIFIEDS
'_02_AR00_02_25834059_jl_ANNOTATED_MISCLASSIFIED_CS.pdf',  '_02_AR03_01_25261920_jl_ANNOTATED_MISCLASSIFIED_SSP.pdf', '_02_AR03_03_25160677_jl_ANNOTATED_MISCLASSIFIED_SSPorCS.pdf', 

In [72]:
import os
# manually annotate these

print(set(possible).intersection(set(stranded)))
print
todo=!ls /Users/ccarey/Documents/Projects/NAMI/rdoc/pdfs/all_pdfs_annotated_pmid_names/_*jl_*
print([os.path.basename(x) for x in todo])

# a non-pubmed document
print
print(set(possible).difference(set(processed).union(set(stranded))))

set(['24804717', '25834059', '25788679', '20685988', '25160677', '25913552', '24806675', '25261920'])

['_02_AP07_02_24804717_jl_ANNOTATED.pdf', '_02_AR00_02_25834059_jl_ANNOTATED_MISCLASSIFIED_CS.pdf', '_02_AR00_03_25788679_jl_ANNOTATED.pdf', '_02_AR03_01_25261920_jl_ANNOTATED_MISCLASSIFIED_SSP.pdf', '_02_AR03_03_25160677_jl_ANNOTATED_MISCLASSIFIED_SSPorCS.pdf', '_02_LS00_03_25913552_jl_IRRELEVANT.pdf', '_02_LS05_02_20685988_jl_ANNOTATED.pdf', '_03_AR03_24806675_jl_ANNOTATED.pdf']

set(['../pdfs/all_pdfs_annotated_pmid_names/02_AG00_01_NA1_jl_IRRELEVANT.pdf'])


In [73]:
# finally cleanup the remaining irrelevant _jl.org 
!rm *_jl.org 

rm: *_jl.org: No such file or directory


In [ ]:
! for f in *[0-9].org; do cp $f ${f/./_tc.}; done

## Note, most of Tara's IRRELEVANTs are blanks

Summary Tara

- TC: 28 stranded
- TC: 45 processed
- TC: 51 possible

Manually procesing 6

- _02_AR00_02_25834059_tc_ANNOTATED.pdf 
- _02_AR03_03_25160677_tc_ANNOTATED.pdf
- _02_AR00_03_25788679_tc_ANNOTATED.pdf
- _02_LS05_02_20685988_tc_ANNOTATED.pdf
- _02_AR03_01_25261920_tc_ANNOTATED.pdf
- _03_AR03_24806675_tc_ANNOTATED.pdf

Leaves 22 stranded

Note: no 'misclassifieds' or other odd ones like in JL set.

In [78]:
! for f in *[0-9].org; do cp $f ${f/./_mk.}; done

Summary Matt

- MK: 23 stranded
- MK: 50 processed
- MK: 60 possible

Manually procesing 8

- _02_AP07_02_24804717_mk_ANNOTATED.pdf
- _02_AR00_02_25834059_mk_ANNOTATED.pdf
- _02_AR00_03_25788679_mk_ANNOTATED.pdf
- _02_AR03_01_25261920_mk_ANNOTATED.pdf
- _02_AR03_03_25160677_mk_ANNOTATED.pdf
- _02_LS00_03_25913552_mk_ANNOTATED_IRRELEVANT.pdf
- _02_LS05_02_20685988_mk_ANNOTATED.pdf
- _03_AR03_24806675_mk_ANNOTATED.pdf

Leaves 15 stranded

Note: no 'misclassifieds' or other odd ones like in JL set.

In [ ]:
! for f in *[0-9].org; do cp $f ${f/./_cc.}; done